In [1]:
import pandas as pd
from utils import connection as con

In [2]:
conn_retail_db = con.connect_retail_db()
conn_dw_retail = con.connect_dw_retail()

In [3]:
from etl import extract as ex

In [5]:
df_customers = ex.extract_table("customers", conn_retail_db)
df_orders = ex.extract_table("orders", conn_retail_db)
df_order_items = ex.extract_table("order_items", conn_retail_db)
df_products = ex.extract_table("products", conn_retail_db)
df_categories = ex.extract_table("categories", conn_retail_db)
df_departments = ex.extract_table("departments", conn_retail_db)


In [6]:
from etl import load

In [8]:
load.load_dw_retail('dimension_customer',conn_dw_retail,df_customers.drop('customer_password', axis=1))

In [11]:
load.load_dw_retail('dimension_department',conn_dw_retail, df_departments)

In [16]:
load.load_dw_retail('dimension_category',conn_dw_retail, df_categories)

In [17]:
load.load_dw_retail('dimension_product',conn_dw_retail, df_products)

In [18]:
load.load_dw_retail('dimension_order',conn_dw_retail, df_orders)

In [19]:
load.load_dw_retail('dimension_order_item',conn_dw_retail, df_order_items)

In [21]:
df=pd.read_sql("SELECT * FROM dimension_order_item", con=conn_dw_retail)
df.head()

,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
0,1,1,957,1,299.98,299.98
1,2,2,1073,1,199.99,199.99
2,3,2,502,5,250.00,50.00
3,4,2,403,1,129.99,129.99
4,5,4,897,2,49.98,24.99


In [22]:
from etl import transform as t

In [ ]:
query = """
SELECT
    DISTINCT DATE_FORMAT(order_date, '%Y%m%d') AS time_id,
    YEAR(order_date) AS year,
    QUARTER(order_date) AS quarter,
    MONTH(order_date) AS month,
    DAY(order_date) AS day,
    DAYOFWEEK(order_date) AS day_of_week,
    DAYOFYEAR(order_date) AS day_of_year,
    WEEK(order_date) AS week_of_year,
    CASE WHEN DAYOFWEEK(order_date) IN (1,7) THEN TRUE ELSE FALSE END AS is_weekend
FROM retail_db.orders
"""

df_dimencion_time = t.transform_time_from_order(query,conn_retail_db)